### trade_record 及 Stockinfo

In [1]:
import pandas as pd
import numpy as np
import os
# connecting SQL 
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0}; SERVER=10.220.8.32; DATABASE=PacketTimeStamp; UID=ptuser; PWD=@Yuanta')

In [2]:
pd.options.display.float_format = '{:.22f}'.format
pd.set_option('display.max_columns', None)

#### **<font color=#FF0000>改日期</font>**、<font color=#00FFFF>確認trade_record 及 stockinfo</font>

In [3]:
trade_record = pd.read_excel('我方成交/20230725.xlsx') # 修改交易資料檔名
date = '20230725' # 修改日期(跟交易資料同一天)

# 跑超級久是正常的(最多曾連續跑3天)、0703, 0710, 0717 是空的
trade_record = trade_record[['Seq','OrderNo', 'TimeStamp', 'Pid', 'Side', 'Price', 'Qty']]

In [4]:
stockinfo = pd.read_csv('stockinfo.csv', encoding= 'big5')
stockinfo.columns = ['Pcode', 'ProductID', '標準型證券股數']

In [5]:
trade_record = trade_record.drop_duplicates(subset=["Seq"], keep='first')
trade_record = trade_record.dropna()
trade_record = trade_record.reset_index()
trade_record['Price'] = trade_record['Price']/100

In [6]:
l1 = []
for i in range(len(trade_record)):
    l1.append(trade_record.loc[i, 'Pid'][:2])
trade_record['Pcode'] = l1

#### 把 stockinfo 的資訊貼到 trade_record

In [7]:
stockinfo = stockinfo.dropna()
stockinfo['ProductID'] = stockinfo['ProductID'].astype(int).astype(str)

In [8]:
trade_record2 = trade_record.merge(stockinfo, on=['Pcode'])

### 連接 TickData 並且貼到 trade_record

#### <font color=#00FFFF>抓下對應的TickData</font>

如果每一次都要撈會很慢，所以最好存起來

In [ ]:
# """
TickData = pd.DataFrame()
for i in range(len(trade_record2)):
    query = f"SELECT Top 1 [SeqNumber], [TimeStamp], [Pid], [BidQty1], [AskQty1], [Bid1], [Ask1] FROM [PacketTimeStamp].[dbo].[View_TaifexRealtime_TickData_{date}] where Format= 'I081' AND Pid = '{trade_record2['Pid'][i]}' AND TimeStamp < {trade_record2['TimeStamp'][i]-0.03} ORDER BY TimeStamp desc"
    TickData = pd.concat([TickData, pd.read_sql(query, conn)], axis = 0) #get SQL data about those stocks (grouped by day)
if not os.path.isdir('期貨資料'):
   os.makedirs('期貨資料')
TickData.to_csv(f'期貨資料/TickData{date}.csv')
#"""

In [9]:
#TickData = pd.read_csv(f'期貨資料/TickData{date}.csv')

#### 貼到 trade_record

In [10]:
trade_record2 = trade_record2.sort_values(['TimeStamp', 'Pid'])
TickData = TickData.sort_values(['TimeStamp', 'Pid'])
TickData = TickData.reset_index(drop=True)
trade_record2 = trade_record2.reset_index(drop=True)

In [11]:
TickData['TimeStamp'] = TickData['TimeStamp'].astype(float)
trade_record3 = pd.merge_asof(trade_record2.sort_values('TimeStamp'), TickData.sort_values('TimeStamp'), on = 'TimeStamp',by = 'Pid')
trade_record3.sort_values(['Pid','TimeStamp']).head(10)

,index,Seq,OrderNo,TimeStamp,Pid,Side,Price,Qty,Pcode,ProductID,標準型證券股數,Unnamed: 0,SeqNumber,BidQty1,AskQty1,Bid1,Ask1
0,40,2,mJ001,1690160400.3348100185394287109375,CAFH3,2,70.5999999999999943156581,1,CA,1303,2000,0,225600,1,3,70.5999999999999943156581,72.0000000000000000000000
83,643,92,mt00p,1690160462.2148399353027343750000,CBFH3,2,29.3500000000000014210855,1,CB,2002,2000,0,705491,1,3,29.3500000000000014210855,29.3999999999999985789145
1025,1770,1554,mt0Ci,1690163439.4947500228881835937500,CBFH3,2,29.3500000000000014210855,2,CB,2002,2000,0,6150918,40,48,29.3000000000000007105427,29.3999999999999985789145
1204,2325,1900,mt0FY,1690165270.8668398857116699218750,CBFH3,2,29.3999999999999985789145,1,CB,2002,2000,0,8422321,2,26,29.3999999999999985789145,29.4499999999999992894573
1212,1976,1915,mt0Fg,1690165380.3573000431060791015625,CBFH3,2,29.3999999999999985789145,1,CB,2002,2000,0,8538854,9,7,29.3500000000000014210855,29.4499999999999992894573
1222,1934,1969,mt0G8,1690165515.2433400154113769531250,CBFH3,2,29.3999999999999985789145,1,CB,2002,2000,0,8671500,1,6,29.3999999999999985789145,29.4499999999999992894573
1306,2098,2161,mt0Hj,1690166970.3294401168823242187500,CBFH3,2,29.3999999999999985789145,2,CB,2002,2000,0,9894074,3,2,29.3999999999999985789145,29.4499999999999992894573
1307,2143,2162,mt0Hk,1690166970.9428999423980712890625,CBFH3,2,29.3999999999999985789145,1,CB,2002,2000,0,9894401,1,3,29.3999999999999985789145,29.4499999999999992894573
453,1125,2106,mG0HD,1690161171.1212699413299560546875,CEFH3,2,63.0000000000000000000000,2,CE,2881,2000,0,2210172,7,4,63.0000000000000000000000,63.1000000000000014210855
454,1126,2107,mG0HE,1690161171.1212699413299560546875,CEFI3,2,63.1000000000000014210855,2,CE,2881,2000,0,2208075,5,6,63.1000000000000014210855,63.3999999999999985789145


In [12]:
def f_available(row):
    if row['Side'] == 1:
        val = row['AskQty1'] - row['Qty']
    else:
        val = row['BidQty1'] - row['Qty']
    return val

trade_record3['future_available'] = trade_record3.apply(f_available, axis=1)

### 連接 TWSEColo 並且貼到 trade_record

#### <font color=#00FFFF>抓下對應的TWSEColo</font>

如果每一次都要撈會很慢，所以最好存起來

In [ ]:
#"""
TWSEColo = pd.DataFrame()
for i in range(len(trade_record3)):
    query2 = f"SELECT TOP 1 [SeqNumber], [TimeStamp], [ProductID], [BidVolume1], [AskVolume1], [Bid1], [Ask1] FROM [PacketTimeStamp].[dbo].[TWSE_Format6_TWSEColo_{date}] WHERE ProductID = '{trade_record2['ProductID'][i]}' AND TimeStamp < {trade_record2['TimeStamp'][i]-0.03} ORDER BY TimeStamp desc"
    TWSEColo = pd.concat([TWSEColo, pd.read_sql(query2, conn)], axis = 0) #get SQL data about those stocks (grouped by day)
if not os.path.isdir('現貨資料'):
   os.makedirs('現貨資料')
TWSEColo.to_csv(f'現貨資料/TWSEColo{date}.csv')
#"""

In [15]:
#TWSEColo = pd.read_csv(f'現貨資料/TWSEColo{date}.csv')

#### 貼到 trade_record

In [16]:
TWSEColo = TWSEColo.sort_values('TimeStamp').sort_values('ProductID')
TWSEColo['TimeStamp'] = TWSEColo['TimeStamp'].astype(float)
TWSEColo['ProductID'] = TWSEColo['ProductID'].apply(str)
TWSEColo = TWSEColo.reset_index(drop=True)
trade_record3 = trade_record3.reset_index(drop=True)
trade_record4 = pd.merge_asof(trade_record3.sort_values('TimeStamp'), TWSEColo.sort_values('TimeStamp'), on='TimeStamp', by = 'ProductID')
trade_record4 = trade_record4.sort_values('TimeStamp').sort_values('Pid').reset_index(drop=True)

In [20]:
#trade_record4 = trade_record4.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis = 1)

In [21]:
trade_record4.columns = ['index', 'Seq', 'OrderNo', 'TimeStamp', 'Pid', 'Side', 'Price', 'Qty',
       'Pcode', 'ProductID', '標準型證券股數','Seq_f',
       'BidQ1_f', 'AskQ1_f', 'Bid1_f', 'Ask1_f', 'future_available',
       'Seq_s', 'BidQ1_s', 'AskQ1_s', 'Bid1_s', 'Ask1_s']
trade_record4 = trade_record4[['Seq', 'OrderNo', 'TimeStamp', 'Pid', 'Side', 'Price', 'Qty',
       'Pcode', 'Seq_f', 'BidQ1_f', 'AskQ1_f', 'Bid1_f', 'Ask1_f',
       'Seq_s', 'BidQ1_s', 'AskQ1_s', 'Bid1_s', 'Ask1_s', 'future_available', '標準型證券股數']]

In [22]:
trade_record4['Bid1_s'] = trade_record4['Bid1_s']/10000
trade_record4['Ask1_s'] = trade_record4['Ask1_s']/10000

In [23]:
def s_available(row):
    if row['Side'] == 1:
        val = row['BidQ1_s'] - row['Qty']*row['標準型證券股數']/1000
    else:
        val = row['AskQ1_s'] - row['Qty']*row['標準型證券股數']/1000
    return val

trade_record4['Stock_available'] = trade_record4.apply(s_available, axis=1)
trade_record4.sort_values('Pid').head(10)

,Seq,OrderNo,TimeStamp,Pid,Side,Price,Qty,Pcode,Seq_f,BidQ1_f,AskQ1_f,Bid1_f,Ask1_f,Seq_s,BidQ1_s,AskQ1_s,Bid1_s,Ask1_s,future_available,標準型證券股數,Stock_available
0,2,mJ001,1690160400.3348100185394287109375,CAFH3,2,70.5999999999999943156581,1,CA,225600,1,3,70.5999999999999943156581,72.0000000000000000000000,NaN,NaN,NaN,NaN,NaN,0,2000,NaN
1,2162,mt0Hk,1690166970.9428999423980712890625,CBFH3,2,29.3999999999999985789145,1,CB,9894401,1,3,29.3999999999999985789145,29.4499999999999992894573,NaN,NaN,NaN,NaN,NaN,0,2000,NaN
2,2161,mt0Hj,1690166970.3294401168823242187500,CBFH3,2,29.3999999999999985789145,2,CB,9894074,3,2,29.3999999999999985789145,29.4499999999999992894573,NaN,NaN,NaN,NaN,NaN,1,2000,NaN
3,92,mt00p,1690160462.2148399353027343750000,CBFH3,2,29.3500000000000014210855,1,CB,705491,1,3,29.3500000000000014210855,29.3999999999999985789145,NaN,NaN,NaN,NaN,NaN,0,2000,NaN
4,1969,mt0G8,1690165515.2433400154113769531250,CBFH3,2,29.3999999999999985789145,1,CB,8671500,1,6,29.3999999999999985789145,29.4499999999999992894573,NaN,NaN,NaN,NaN,NaN,0,2000,NaN
5,1554,mt0Ci,1690163439.4947500228881835937500,CBFH3,2,29.3500000000000014210855,2,CB,6150918,40,48,29.3000000000000007105427,29.3999999999999985789145,NaN,NaN,NaN,NaN,NaN,38,2000,NaN
6,1900,mt0FY,1690165270.8668398857116699218750,CBFH3,2,29.3999999999999985789145,1,CB,8422321,2,26,29.3999999999999985789145,29.4499999999999992894573,NaN,NaN,NaN,NaN,NaN,1,2000,NaN
7,1915,mt0Fg,1690165380.3573000431060791015625,CBFH3,2,29.3999999999999985789145,1,CB,8538854,9,7,29.3500000000000014210855,29.4499999999999992894573,NaN,NaN,NaN,NaN,NaN,8,2000,NaN
8,2106,mG0HD,1690161171.1212699413299560546875,CEFH3,2,63.0000000000000000000000,2,CE,2210172,7,4,63.0000000000000000000000,63.1000000000000014210855,NaN,NaN,NaN,NaN,NaN,5,2000,NaN
9,2107,mG0HE,1690161171.1212699413299560546875,CEFI3,2,63.1000000000000014210855,2,CE,2208075,5,6,63.1000000000000014210855,63.3999999999999985789145,NaN,NaN,NaN,NaN,NaN,3,2000,NaN


### 結果

In [24]:
def trade_more(row):
    if row['future_available'] > 0 and row['Stock_available'] > 0:
        a = row['future_available']
        b = row['Stock_available']//(row['標準型證券股數']/1000)
        val = min(a, b)
    else:
        val = 'NA'
    return val
trade_record4['more_f'] = trade_record4.apply(trade_more, axis=1)

In [25]:
trade_record4['Ask1_f'] = trade_record4['Ask1_f'].astype(float)
trade_record4['Bid1_f'] = trade_record4['Bid1_f'].astype(float)
trade_record4['Ask1_s'] = trade_record4['Ask1_s'].astype(float)
trade_record4['Bid1_s'] = trade_record4['Bid1_s'].astype(float)

#### 我方交易

In [26]:
def tick_cal_our(row):
    val = row['Price']*row['Qty']*row['標準型證券股數']/10000
    return  val

trade_record4['result_our'] = trade_record4.apply(tick_cal_our, axis=1)

In [27]:
def ratio_cal_our(row):
    try:
        if row['Side'] == 1:
            val = row['Price']/row['Bid1_s'] - 1
        else:
            val = row['Price']/row['Ask1_s'] - 1
    except:
        val = '分母為零'
    return  val

trade_record4['ratio_our'] = trade_record4.apply(ratio_cal_our, axis=1)

#### 市場剩餘

In [28]:
def tick_cal_M(row):
    if row['more_f'] == 'NA':
        return
    if row["Side"] == 1:
        val = row['Ask1_f']*row['more_f']/10
        
    else:
        val = row['Bid1_f']*row['more_f']/10
    return  val

trade_record4['result_Market'] = trade_record4.apply(tick_cal_M, axis=1)

In [29]:
def ratio_cal_M(row):
    try:
        if row['Side'] == 1:
            val = row['Ask1_f']/row['Bid1_s'] - 1
        else:
            val = row['Bid1_f']/row['Ask1_s'] - 1
    except:
        val = '分母為零'
    return  val

trade_record4['ratio_M'] = trade_record4.apply(ratio_cal_M, axis=1)

In [32]:
trade_record4 = trade_record4.dropna()

In [33]:
trade_record4.to_excel('cal_remain_proccess.xlsx')

#### <font color=#FFD700>**名詞解釋與成果輸出:**</font>
- future_available: 交易前期貨市場的存量(BidQ1_f or AskQ1_f) - 交易數量(Qty)
- Stock_available: 交易前現貨市場的存量(BidQ1_s or AskQ1_s) - 交易數量(Qty)
- more_f: 考慮剩餘的期貨與現貨後，可再增加套利的期貨數量 (min(future_available, 證券轉換Stock_available))
- result: (以萬為單位)
    - Our: 'Price' * 'Qty' * '標準型證券股數'/10000
    - Market: ('Ask1_f' or 'Bid1_f') * 'more_f'/10
- ratio: ('Ask1_f'/'Bid1_s') - 1 if side == 1

#### 按 tick 分類的需求

In [326]:
def diff(row):
    if row['Side'] == 1:
        return row['Price'] - row['Bid1_s']
    elif row['Side'] == 2:
        return row['Price'] - row['Ask1_s']
trade_record4['diff'] = trade_record4.apply(diff, axis=1)
def which_tick(row):
    if row['Price'] < 10:
        return 0.01
    elif 10 < row['Price'] <= 50:
        return 0.05
    elif 50 < row['Price'] <= 100:
        return 0.1
    elif 100 < row['Price'] <= 500:
        return 0.5
    elif 500 < row['Price'] <= 1000:
        return 1
    elif 1000 < row['Price']:
        return 5
trade_record4['tick'] = trade_record4.apply(which_tick, axis=1)
def num_tick(row):
    if pd.isnull(row['diff']):
        return
    return round(row['diff']/row['tick'])
trade_record4['num_tick'] = trade_record4.apply(num_tick, axis=1)
trade_record4 = trade_record4.sort_values(['Side', 'Pid', 'TimeStamp'])

In [327]:
result_list = []
result_list.append(trade_record4.groupby(['Side', 'Pid', 'num_tick'])['result_Market'].sum())
result_list.append(trade_record4.groupby(['Side', 'Pid', 'num_tick'])['result_our'].sum())
result_df = pd.concat(result_list, axis = 1).reset_index()
# result_df.insert(0, '日期', date)
result_df.columns = ['Side', 'Pid','num_tick', 'result_Market', 'result_our']

In [328]:
if not os.path.isdir('查詢結果'):
   os.makedirs('查詢結果')
result_df.to_excel(f'查詢結果/result_sum_{date}.xlsx')

#### Sheet2

In [329]:
def aaaaa(x):
    # x 當前兩個條件小組的dataframe
    if len(x) == 1:
        return x.iloc[0, 3], x.iloc[0, 4]
    elif len(x)%2 == 0:
        mid = int(len(x)/2) # iloc 不能有小數點
        return min(x.iloc[0:mid, 3].sum(), x.iloc[mid:, 3].sum()), min(x.iloc[0:mid, 4].sum(), x.iloc[mid:, 4].sum())
    elif len(x)%2 == 1:
        mid = len(x)//2 # iloc 不能有小數點
        a = min(x.iloc[0:mid, 3].sum() + x.iloc[mid, 3], x.iloc[mid+1:, 3].sum())
        b = min(x.iloc[0:mid, 3].sum() , x.iloc[mid+1:, 3].sum() + x.iloc[mid, 3])
        c = min(x.iloc[0:mid, 4].sum() + x.iloc[mid, 4], x.iloc[mid+1:, 4].sum())
        d = min(x.iloc[0:mid, 4].sum() , x.iloc[mid+1:, 4].sum() + x.iloc[mid, 4])
        return max(a, b), max(c, d)

In [330]:
from openpyxl import load_workbook
filename = f'查詢結果/result_sum_{date}.xlsx'

df = pd.read_excel(filename)
df = df.iloc[:, 1:]
result = df.groupby(['Side', 'Pid']).apply(aaaaa)
temp = pd.DataFrame(pd.DataFrame(result)[0].tolist(), index=result.index)
result = pd.concat([result, temp], axis = 1)
result = result.iloc[:, 1:]
result.columns = ['result_Market', 'result_our']


book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book
result.to_excel(writer, sheet_name = 'new result')
writer.close()

##### Sheet3

In [ ]:
filename = f'查詢結果/result_sum_{date}.xlsx'
df = pd.read_excel(filename)
df = df.iloc[:, 1:]
df2 = df.groupby(['Side', 'Pid'])['result_Market'].sum()
df2 = df2.reset_index()
df2 = df2.set_index('Side', drop = True)
def asd(row):
    if len(row['result_Market']) == 2:
        return row['result_Market']
    else:
        return
ss = df2.groupby('Pid').apply(asd)
ss = ss.dropna()
def asdd(row):
    return min(row[1], row[2])
ss.insert(0, 'remain amount(萬元)', ss.apply(asdd, axis = 1))

ss_final = ss[ss['remain amount(萬元)']!=0]['remain amount(萬元)'].sort_values(ascending = False)

book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book
ss_final.to_excel(writer, sheet_name = 'new result2')
writer.close()

#### 查資料

In [331]:
trade_record4[trade_record4['Seq']==1870]

,Seq,OrderNo,TimeStamp,Pid,Side,Price,Qty,Pcode,Seq_f,BidQ1_f,AskQ1_f,Bid1_f,Ask1_f,Seq_s,BidQ1_s,AskQ1_s,Bid1_s,Ask1_s,future_available,標準型證券股數,Stock_available,more_f,result_our,ratio_our,result_Market,ratio_M,diff,tick,num_tick
1859,1870,mG0FI,1690161124.6572699546813964843750,RJFH3,2,40.8999999999999985789145,1,RJ,2125206,2,5,40.8999999999999985789145,40.9500000000000028421709,NaN,NaN,NaN,NaN,NaN,1,2000,NaN,NA,8.1799999999999997157829,NaN,NaN,NaN,NaN,0.0500000000000000027756,NaN


In [332]:
trade_record3[trade_record3['Seq']==1870]

,index,Seq,OrderNo,TimeStamp,Pid,Side,Price,Qty,Pcode,ProductID,標準型證券股數,SeqNumber,BidQty1,AskQty1,Bid1,Ask1,future_available
434,969,1870,mG0FI,1690161124.6572699546813964843750,RJFH3,2,40.8999999999999985789145,1,RJ,1609,2000,2125206,2,5,40.90,40.95,1
